In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from matplotlib.patches import Circle
from scipy.spatial import distance
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder
from photutils import CircularAperture
import sys
# sys.path.insert(1,'/home/karina/Dropbox/postdoc_EPFL/ANN/CNNstudentproject/CNN_200k/classification_mosaic/known_candidates/catalogs/')
import VI_def as vid
import progressbar
from astropy.table import QTable
from os import listdir
from os.path import isfile, join, isdir, exists
import re

In [26]:
path = '/Users/markmaus/Desktop/Physics_EPFL/Specialization_Project/lens_candidates/Group1/'
im_path = '/Users/markmaus/Desktop/Physics_EPFL/Specialization_Project/lens_candidates/Group1/data/'

In [37]:
obj_name_location = 0
deltaPix = 0.27

In [28]:
im_files = [f for f in listdir(im_path) if isfile('/'.join([im_path,f]))]

obj_names = []
for x in im_files:
    obj_names.append(re.findall('\d+', x)[obj_name_location])
obj_names = sorted(obj_names,key=float)
obj_names

['00079820',
 '00133061',
 '00688991',
 '01873198',
 '02171611',
 '02425798',
 '02871220',
 '04187953',
 '04357745',
 '05336690',
 '05377650',
 '05500839',
 '05559683',
 '06653211',
 '06788344',
 '07087938',
 '07417737',
 '08037325',
 '10677910',
 '11308901',
 '12556054',
 '12670631',
 '12746662',
 '13242988',
 '13671908',
 '14083401',
 '14327423',
 '14635130',
 '14731625',
 '15172960',
 '15308357',
 '15536213',
 '15977522',
 '16033319',
 '17103670',
 '17261494',
 '17640721',
 '17856142',
 '19368638',
 '19566497',
 '19990514',
 '20366791',
 '20530547',
 '20906435',
 '23636368',
 '23998704',
 '25227807',
 '25315733',
 '25559914']

In [29]:
len(obj_names)

49

In [30]:
df = pd.read_csv(path+'group_1.csv')
# df_2 = pd.DataFrame(df.loc[0]).transpose()
# df_2

In [32]:
df.loc[df['id_1'].isin(obj_names)]



,Unnamed: 0,level_0,Unnamed: 0.1,col1_1,col1_1a,ra,dec,id_1,names,references,...,sum,SL_per,class_fVI_2,subclass_fVI,comments_fVI,g-i,g-r,rad,decd,group
0,0,72,74,74,74,4.818033,-41.614051,79820,DESJ0019-4136,Jacobs2019b,...,10,0.571429,L,None,NaN,2.0119,1.3980,345.550175,4.899655,g1
1,2,17,18,18,18,9.362803,-41.530542,133061,DESJ0037-4131,Diehl2017 / Jacobs2019b,...,19,1.000000,L,None,NaN,2.2036,1.3135,341.005405,4.983164,g1
2,4,79,81,81,81,9.593161,-25.842242,688991,DESJ0038-2550,Jacobs2019a / Jacobs2019b,...,6,0.428571,L,None,NaN,2.1802,1.4190,340.775047,20.671464,g1
3,6,0,0,0,0,3.928313,-46.603047,1873198,DESJ0015-4636,Jacobs2019b,...,21,1.000000,L,None,NaN,2.0101,1.3587,346.439895,0.089341,g1
4,7,49,51,51,51,18.389652,-38.220203,2171611,NaN,NaN,...,16,0.857143,L,None,NaN,2.3868,1.2201,331.978556,8.293503,g1
5,8,1,1,1,1,15.366041,-33.722010,2425798,IACJ010127.83-334319.6 / DESJ0101-3343 / LinKS...,Bettinelli2016 / Jacobs2019b / Petrillo2019,...,21,1.000000,L,None,NaN,1.8416,1.0854,335.002167,12.791696,g1
6,9,54,56,56,56,19.194939,-24.617243,2871220,DESJ0116-2437,Jacobs2019b,...,15,0.857143,L,None,NaN,1.9163,1.1972,331.173269,21.896463,g1
7,11,55,57,57,57,15.491818,-49.293942,4187953,DESJ0101-4917,Jacobs2019a / Jacobs2019b,...,15,0.857143,L,None,NaN,2.2177,1.0965,334.876390,2.780236,g1
8,12,24,25,25,25,16.746389,-44.533731,4357745,DESJ0106-4432,Jacobs2019b,...,18,1.000000,L,None,NaN,1.8278,1.0706,333.621819,1.979975,g1
9,17,38,40,40,40,25.720295,-18.521051,5336690,DESJ0142-1831,Jacobs2019a / Jacobs2019b,...,17,0.857143,L,None,NaN,2.3046,1.3556,324.647913,27.992655,g1


In [45]:
test_id = '00000133061'
ra = float(df.loc[df['id_1'] == int(test_id),'ra'])
ra

9.362803

In [33]:
df['dst_arcsec'] = 100
df['dst_pix'] = 100
df

,Unnamed: 0,level_0,Unnamed: 0.1,col1_1,col1_1a,ra,dec,id_1,names,references,...,class_fVI_2,subclass_fVI,comments_fVI,g-i,g-r,rad,decd,group,dst_arcsec,dst_pix
0,0,72,74,74,74,4.818033,-41.614051,79820,DESJ0019-4136,Jacobs2019b,...,L,None,NaN,2.0119,1.3980,345.550175,4.899655,g1,100,100
1,2,17,18,18,18,9.362803,-41.530542,133061,DESJ0037-4131,Diehl2017 / Jacobs2019b,...,L,None,NaN,2.2036,1.3135,341.005405,4.983164,g1,100,100
2,4,79,81,81,81,9.593161,-25.842242,688991,DESJ0038-2550,Jacobs2019a / Jacobs2019b,...,L,None,NaN,2.1802,1.4190,340.775047,20.671464,g1,100,100
3,6,0,0,0,0,3.928313,-46.603047,1873198,DESJ0015-4636,Jacobs2019b,...,L,None,NaN,2.0101,1.3587,346.439895,0.089341,g1,100,100
4,7,49,51,51,51,18.389652,-38.220203,2171611,NaN,NaN,...,L,None,NaN,2.3868,1.2201,331.978556,8.293503,g1,100,100
5,8,1,1,1,1,15.366041,-33.722010,2425798,IACJ010127.83-334319.6 / DESJ0101-3343 / LinKS...,Bettinelli2016 / Jacobs2019b / Petrillo2019,...,L,None,NaN,1.8416,1.0854,335.002167,12.791696,g1,100,100
6,9,54,56,56,56,19.194939,-24.617243,2871220,DESJ0116-2437,Jacobs2019b,...,L,None,NaN,1.9163,1.1972,331.173269,21.896463,g1,100,100
7,11,55,57,57,57,15.491818,-49.293942,4187953,DESJ0101-4917,Jacobs2019a / Jacobs2019b,...,L,None,NaN,2.2177,1.0965,334.876390,2.780236,g1,100,100
8,12,24,25,25,25,16.746389,-44.533731,4357745,DESJ0106-4432,Jacobs2019b,...,L,None,NaN,1.8278,1.0706,333.621819,1.979975,g1,100,100
9,17,38,40,40,40,25.720295,-18.521051,5336690,DESJ0142-1831,Jacobs2019a / Jacobs2019b,...,L,None,NaN,2.3046,1.3556,324.647913,27.992655,g1,100,100


In [35]:
bar = progressbar.ProgressBar(maxval=len(df), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()


for j in range(len(df)):
	bar.update(j)
	im= fits.open(im_path+df['file_name'].iloc[j])
	mean, median, std = sigma_clipped_stats(im[0].data, sigma=3.0)
	daofind = DAOStarFinder(fwhm=5.0, threshold=5.*std)
	sources = daofind(im[0].data - median)
	if sources == None:
	    daofind = DAOStarFinder(fwhm=8.0, threshold=5.*std)
	    sources = daofind(im[0].data - median)
	if sources == None:
		x = [0,15]
		y = [0,15]
		sources = QTable([x,y],names={'xcentroid','ycentroid'})
	sources['difx'] = abs(sources['xcentroid']-26)
	sources['dify'] = abs(sources['ycentroid']-24)
	s = sources.to_pandas()
	print(s)
	s['md'] = s[['difx','dify']].mean(axis=1)
	s = s.sort_values(by=['md'])
	s = s.reset_index()	
	if (len(s)>1) and (s['md'][0]<3):
	    print('d1',s['xcentroid'].values[1],s['ycentroid'].values[1])
	    dst=distance.euclidean((25,25),(s['xcentroid'].values[1],s['ycentroid'].values[1]))
	    print(dst)
	    if (s['md'][1]>12):
	        print('d2')
	        dst=distance.euclidean((25,25),(s['xcentroid'].values[0],s['ycentroid'].values[0]))   
	else:
	    print('re-c')
	    daofind = DAOStarFinder(fwhm=8.0, threshold=5.*std)
	    sources = daofind(im[0].data - median)
	    sources['difx'] = abs(sources['xcentroid']-26)
	    sources['dify'] = abs(sources['ycentroid']-24)
	    s = sources.to_pandas()
	    s['md'] = s[['difx','dify']].mean(axis=1)
	    s = s.sort_values(by=['md'])
	    s = s.reset_index()
	    print(s)
	    print(len(s))
	    if (len(s)==1):
	        print('d3')
	        dst=distance.euclidean((25,25),(s['xcentroid'].values[0],s['ycentroid'].values[0]))
	    if (len(s)>1) :
	        if (s['md'][1]>12):
	            print('d4')
	            dst=distance.euclidean((25,25),(s['xcentroid'].values[0],s['ycentroid'].values[0]))
	        else:
	            print('d5')
	            dst=distance.euclidean((25,25),(s['xcentroid'].values[1],s['ycentroid'].values[1]))
	df['dst_pix'].iloc[j] = dst
	df['dst_arcsec'].iloc[j] = dst*deltaPix
bar.finish()

/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  28.036007  23.966919   0.349308   -0.195663   -0.685484    49  0.0   
1   2  25.659157  25.055920   0.529572   -0.407090   -0.008127    49  0.0   

        peak      flux       mag      difx      dify  
0  24.409756  1.199144 -0.197179  2.036007  0.033081  
1  37.487099  1.797193 -0.636487  0.340843  1.055920  
d1 28.036007119481017 23.9669191062278
3.206960455418275


/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value 

   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  35.745303  25.105744   0.298975    0.048111    0.135997    49  0.0   

        peak      flux       mag      difx      dify  
0  12.578761  1.244651 -0.237619  9.745303  1.105744  
re-c
   index  id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  \
0      2   3  27.374333  18.214835   0.648354    0.522766   -0.876638   121   
1      3   4  35.313404  24.788490   0.429659    0.313849   -0.360434   121   
2      1   2  22.455816  17.218449   0.910694   -0.459907   -0.624226   121   
3      0   1  26.065974   1.263076   0.506711   -0.304878   -0.808866   121   
4      4   5  10.437018  39.504582   0.554482   -0.169394   -0.308456   121   

   sky       peak      flux       mag       difx       dify         md  
0  0.0  12.451430  1.229748 -0.224540   1.374333   5.785165   3.579749  
1  0.0  12.183433  1.786831 -0.630208   9.313404   0.788490   5.050947  
2  0.0  16.242577  1.506298 -0.44477

/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value 

...

   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  25.387680  25.933794   0.275794    0.350830   -0.154256    49  0.0   
1   2  23.211908  47.922148   0.681510   -0.705072   -0.400422    49  0.0   

        peak      flux       mag      difx       dify  
0  25.871548  1.502516 -0.442048  0.612320   1.933794  
1  14.843925  1.227160 -0.222253  2.788092  23.922148  
d1 23.211908460410264 47.92214781028804
22.99178400191349
d2
   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  21.436931  15.940565   0.481782   -0.708908   -0.920518    49  0.0   

        peak      flux       mag      difx      dify  
0  11.010995  1.020547 -0.022083  4.563069  8.059435  
re-c
   index  id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  \
0      2   3  25.945994  25.508930   0.342737    0.034422    0.051819   121   
1      0   1  31.877077  18.836756   0.452756    0.915562   -0.241480   121   
2      1   2  13.789621  20.1694

/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value 

   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  24.995094  25.196309   0.566061    0.045996     -0.1541    49  0.0   

        peak      flux       mag      difx      dify  
0  43.729942  2.325061 -0.916086  1.004906  1.196309  
re-c
   index  id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  \
0      0   1  24.963690  25.010962   0.573198   -0.080947   -0.177453   121   
1      1   2  34.453396  26.462930   0.884729    0.193788   -0.393329   121   

   sky       peak      flux       mag      difx      dify        md  
0  0.0  43.729942  4.636991 -1.665591  1.036310  1.010962  1.023636  
1  0.0  13.453988  1.128879 -0.131618  8.453396  2.462930  5.458163  
2
d5
   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  34.882753  25.711514   0.427394   -0.102801    0.547322    49  0.0   
1   2  19.121507  33.004742   0.456302    0.554029    0.357359    49  0.0   
2   3  29.474654  31.807206   0.587979   -0.36

/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/markmaus/anaconda3/lib/python3.7/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other
/Users/markmaus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value 

   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  22.193972  23.973345   0.498764   -0.571482    0.220184    49  0.0   
1   2  25.454540  25.167416   0.751989    0.274198   -0.311123    49  0.0   

        peak      flux       mag      difx      dify  
0  45.160427  3.133450 -1.240057  3.806028  0.026655  
1  24.400154  1.000046 -0.000050  0.545460  1.167416  
d1 22.193972453913783 23.97334520778019
2.9879442183184297
   id  xcentroid  ycentroid  sharpness  roundness1  roundness2  npix  sky  \
0   1  27.139115   9.789495   0.329228   -0.029037   -0.768081    49  0.0   
1   2  25.082691  25.500554   0.406626   -0.005351    0.188506    49  0.0   
2   3  20.652121  33.854384   0.393084   -0.100468   -0.159144    49  0.0   

        peak      flux       mag      difx       dify  
0  22.754360  1.602767 -0.512176  1.139115  14.210505  
1  22.308613  1.556500 -0.480373  0.917309   1.500554  
2  15.160617  1.444793 -0.399514  5.347879   9.854384  
d1 20.65212

In [38]:
bar.finish()

[========================================================================] 100%


In [39]:

df.to_csv(path + 'mask_v2.csv')

#*****************
#	Mosaic
#*****************

df=pd.read_csv(path+'mask_v2.csv')

vid.mosaic_mask(im_path+df['file_name'],12,path+'mask_v2',df['dst_pix'],8,25,var=df['dst_arcsec'],vname='TE = ')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
